In [1]:
import pandas as pd
import seaborn as sns
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error


# Libraries for Forecasting
import pandas as pd
import numpy as np
import warnings
import itertools
import matplotlib
import matplotlib.pyplot as plt
import datetime
from pylab import rcParams
import statsmodels.api as sm
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.stattools import adfuller
from sklearn import preprocessing
from pycaret.regression import *

# Ignore the warning, can comment this 
warnings.filterwarnings("ignore")

# Different Baselines

#### Numero para hacer mas rápido el notebook. (la version final debe llevar número = 100, pero se va a tardar el notebook en correr)

In [2]:
numero = 100

In [3]:
df_repsol = pd.read_csv('df_repsol_wo_FE.csv', index_col=1).drop(columns = ["Unnamed: 0"])
df_repsol["station"] = df_repsol["station"].str[2:]
df_repsol["station"] = df_repsol["station"].astype(int)

In [4]:
le = preprocessing.LabelEncoder()
df_repsol['municipality'] = le.fit_transform(df_repsol["municipality"].values)

In [5]:
df_repsol['producto'] = 0
df_repsol.loc[df_repsol['product'] == '95', 'producto'] = 1

In [6]:
columns_strings = df_repsol.dtypes[df_repsol.dtypes == "object"].index
columns_strings

Index(['product', 'horaracha', 'horatmax', 'horatmin', 'coordinates',
       'provinces'],
      dtype='object')

In [7]:
def r2_rmse(data):
    r2 = r2_score(data['sales_norm'], data['Label'] )
    mae =  mean_absolute_error(data['sales_norm'], data['Label'])
    return pd.Series(dict(mae = mae, r2 = r2))

In [8]:
df_repsol_model = df_repsol.drop(columns = columns_strings)
df_repsol_model.dropna(inplace = True)

In [9]:
global_train = df_repsol_model.loc[(df_repsol_model.index >= '2016-01-01')
                     & (df_repsol_model.index <= '2018-12-31')]

global_test = df_repsol_model.loc[(df_repsol_model.index >= '2019-01-01')
                     & (df_repsol_model.index <= '2019-01-31')]

# 1. Model predictions without Feature Engineering; Global model w/o Feature Engineering using pycaret

We then create a Global Model by testing diferent algorithms such as Lightgbm, Xgboost and Random Forest Regressor

In [10]:
df_repsol_model = df_repsol.drop(columns = columns_strings)
df_repsol_model.dropna(inplace = True)

In [11]:
global_train = df_repsol_model.loc[(df_repsol_model.index >= '2016-01-01')
                     & (df_repsol_model.index <= '2018-12-31')]

global_test = df_repsol_model.loc[(df_repsol_model.index >= '2019-01-01')
                     & (df_repsol_model.index <= '2019-01-31')]

In [12]:
X_train_global = global_train.drop("sales_norm", axis=1)
Y_train_global = global_train["sales_norm"]

X_test_global = global_test.drop("sales_norm", axis=1)
Y_test_global = global_test["sales_norm"]

In [23]:
setup1 = setup(data = global_train,  target = "sales_norm", silent = True) 

,Description,Value
0,session_id,856
1,Target,sales_norm
2,Original Data,"(24992, 36)"
3,Missing Values,False
4,Numeric Features,25
5,Categorical Features,10
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(17494, 49)"


In [24]:
best_global = compare_models(sort="R2", exclude = ["lar", "huber", "par", "llar", "knn", "lasso", "en"])

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
omp,Orthogonal Matching Pursuit,0.1117,2.5802,0.6277,0.5982,0.1005,0.7899,0.0280
ridge,Ridge Regression,0.1103,2.5808,0.6286,0.5941,0.1008,0.7696,0.0290
lr,Linear Regression,0.1112,2.5811,0.6295,0.5891,0.1016,0.7743,0.0390
br,Bayesian Ridge,0.1135,2.5815,0.6309,0.5815,0.1020,0.8115,0.0590
dummy,Dummy Regressor,0.2178,2.6135,0.7165,-0.0044,0.1740,1.4466,0.0220
lightgbm,Light Gradient Boosting Machine,0.0981,2.9074,0.9609,-5.4479,0.1210,0.5957,0.2050
gbr,Gradient Boosting Regressor,0.1257,3.0397,1.0126,-7.8959,0.1373,0.8117,1.3840
rf,Random Forest Regressor,0.0864,3.5683,1.2298,-17.6302,0.1103,0.4900,4.4510
et,Extra Trees Regressor,0.0886,4.4892,1.3496,-34.5870,0.0986,0.4837,3.5300
dt,Decision Tree Regressor,0.0939,5.1409,1.1008,-46.7180,0.0891,0.4971,0.1150


In [25]:
evaluate_model(best_global)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [26]:
predictions_train_global = predict_model(best_global, data= global_train)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Orthogonal Matching Pursuit,0.1080,1.8089,1.3450,0.0202,0.0993,0.7284


In [27]:
predictions_train_global.groupby('producto').apply(r2_rmse).reset_index()

,producto,mae,r2
0,0,0.130456,-0.009635
1,1,0.085587,0.002660


In [28]:
predictions_test_global = predict_model(best_global, data= global_test)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Orthogonal Matching Pursuit,0.1049,0.0214,0.1464,0.4883,0.1083,0.9180


In [29]:
predictions_test_global.groupby('producto').apply(r2_rmse).reset_index()

,producto,mae,r2
0,0,0.140608,-0.294059
1,1,0.069186,0.137668


# 2. Basline Model using the Sales Lag 7

In [196]:
df_repsol2 = pd.read_csv('df_repsol(3).csv', index_col=1).drop(columns = ["Unnamed: 0"])
df_repsol2["station"] = df_repsol2["station"].str[2:]
df_repsol2["station"] = df_repsol2["station"].astype(int)
le = preprocessing.LabelEncoder()
df_repsol2['municipality'] = le.fit_transform(df_repsol2["municipality"].values)
df_repsol2['producto'] = 0
df_repsol2.loc[df_repsol2['product'] == '95', 'producto'] = 1

In [197]:
columns_strings2 = df_repsol2.dtypes[df_repsol2.dtypes == "object"].index
columns_strings2

Index(['product', 'horaracha', 'horatmax', 'horatmin', 'coordinates',
       'provinces'],
      dtype='object')

In [198]:
def r2_rmse(data):
    r2 = r2_score(data['sales_norm'], data['Label'] )
    mae =  mean_absolute_error(data['sales_norm'], data['Label'])
    return pd.Series(dict(mae = mae, r2 = r2))

In [199]:
df_repsol_model2 = df_repsol2.drop(columns = columns_strings)
df_repsol_model2.dropna(inplace = True)

In [200]:
global_train2 = df_repsol_model2.loc[(df_repsol_model2.index >= '2016-01-01')
                     & (df_repsol_model.index <= '2018-12-31')]

global_test2 = df_repsol_model2.loc[(df_repsol_model2.index >= '2019-01-01')
                     & (df_repsol_model.index <= '2019-01-31')]

In [201]:
df_repsol2[['sales_norm', 'lag7_sales']]

,sales_norm,lag7_sales
date,,
2016-01-01,0.024927,0.024927
2016-01-02,0.064939,0.024927
2016-01-03,0.069706,0.024927
2016-01-04,0.094124,0.024927
2016-01-05,0.097476,0.024927
...,...,...
2019-01-27,0.360448,0.354203
2019-01-28,0.594835,0.620668
2019-01-29,0.546949,0.580848


In [205]:
round(r2_score(global_train2['sales_norm'], global_train2['lag7_sales']),3)

0.623

As expected, the r2 scores for both the training and test set are not very good.

In [209]:
print(f"Training Set R2 score for baseline model: {round(r2_score(global_train['sales_norm'], global_train['lag7_sales']),3)}")

Training Set R2 score for baseline model: 0.623


In [208]:
print(f"Testing Set R2 score for baseline model: {round(r2_score(global_test['sales_norm'], global_test['lag7_sales']),3)}")

Testing Set R2 score for baseline model: 0.632


# 3. Creating a Baseline Model using Linear Regression

Using some more complex methods, we managed to increase the R2 score, this time, using a Linear Regression Model.

In [187]:
X_train_global = global_train2.drop("sales_norm", axis=1)
Y_train_global = global_train2["sales_norm"]

X_test_global = global_test2.drop("sales_norm", axis=1)
Y_test_global = global_test2["sales_norm"]

In [188]:
setup_lr = setup(data = global_train2,  target = "sales_norm", silent = True) 

,Description,Value
0,session_id,880
1,Target,sales_norm
2,Original Data,"(24987, 80)"
3,Missing Values,False
4,Numeric Features,66
5,Categorical Features,13
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(17490, 89)"


#### Pycaret uses a default CV grid of 10 folds (we set it to 100 to get more consistent results), using 70% of the data as training data and 30% for validation

In [189]:
linear_model = create_model("lr", fold = numero)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.0723,0.0132,0.1149,0.7742,0.0822,0.3961
1,0.0683,0.0104,0.1021,0.8112,0.0739,0.5096
2,0.0579,0.0072,0.0847,0.8765,0.0587,0.2243
3,0.0539,0.0058,0.0763,0.8863,0.0517,0.2208
4,0.0741,0.0130,0.1139,0.7880,0.0834,0.5932
5,0.0567,0.0066,0.0809,0.8846,0.0561,0.2431
6,0.0667,0.0087,0.0933,0.8329,0.0646,0.2611
7,0.0602,0.0081,0.0898,0.8569,0.0625,0.2460
8,0.0671,0.0111,0.1055,0.8000,0.0755,0.4101


In [190]:
predictions_train = predict_model(linear_model, data= global_train2)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,0.0647,0.0090,0.0947,0.8351,0.0666,0.4477


In [191]:
predictions_test = predict_model(linear_model, data= global_test2)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Linear Regression,0.0684,0.0095,0.0976,0.7729,0.0738,0.5820


#### Grouping by product, we can see the model struggling to predict the GOA sales the most as it gets an R2 of 24% in the test set.

In [192]:
predictions_train.groupby('producto').apply(r2_rmse).reset_index()

,producto,mae,r2
0,0,0.094060,0.438347
1,1,0.035378,0.769300


In [193]:
predictions_test.groupby('producto').apply(r2_rmse).reset_index()

,producto,mae,r2
0,0,0.092289,0.425676
1,1,0.044583,0.616747
